In [63]:
#Load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import wordnet
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import Levenshtein

import os
print(os.listdir("../SentimentAnalysis/input"))
import warnings
warnings.filterwarnings('ignore')


['IMDB Dataset.csv']


In [84]:
norm_train_reviews=pd.read_csv('../SentimentAnalysis/normal_train_reviews.csv')['review']

norm_test_reviews=pd.read_csv('../SentimentAnalysis/normal_test_reviews.csv')['review']

augmented_train_reviews=pd.read_csv('../SentimentAnalysis/augmented_train_reviews.csv')['review']

augmented_test_reviews=pd.read_csv('../SentimentAnalysis/augmented_test_reviews.csv')['review']

augmented_train_reviews_p=pd.read_csv('../SentimentAnalysis/augmented_train_reviews_p.csv')['review']

augmented_test_reviews_p=pd.read_csv('../SentimentAnalysis/augmented_test_reviews_p.csv')['review']

#importing the training data
imdb_data=pd.read_csv('../SentimentAnalysis/input/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [65]:
print(norm_train_reviews.shape)
print(norm_test_reviews.shape)
print(augmented_train_reviews.shape)
print(augmented_test_reviews.shape)


(40000,)
(10000,)
(40000,)
(10000,)


In [66]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0.0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviewsn=cv.fit_transform(norm_train_reviews)

#transformed test reviews
cv_test_reviewsn=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviewsn.shape)
print('BOW_cv_test:',cv_test_reviewsn.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (40000, 6209089)
BOW_cv_test: (10000, 6209089)


In [87]:
cv1=CountVectorizer(min_df=0.0,max_df=1,binary=False,ngram_range=(1,3), stop_words = None)

cv_train_reviewsa=cv1.fit_transform(augmented_train_reviews)

cv_test_reviewsa=cv1.transform(augmented_test_reviews)


In [88]:
cv2=CountVectorizer(min_df=0.0,max_df=1,binary=False,ngram_range=(1,3), stop_words = None)

cv_train_reviewsap=cv2.fit_transform(augmented_train_reviews_p)

cv_test_reviewsap=cv2.transform(augmented_test_reviews_p)


In [68]:
cv_test_reviewsa.shape

(10000, 6055854)

In [69]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0.0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviewsn=tv.fit_transform(norm_train_reviews)

#transformed test reviews
tv_test_reviewsn=tv.transform(norm_test_reviews)

print('Tfidf_train:',tv_train_reviewsn.shape)
print('Tfidf_test:',tv_test_reviewsn.shape)

Tfidf_train: (40000, 6209089)
Tfidf_test: (10000, 6209089)


In [70]:
tv1=TfidfVectorizer(min_df=0.0,max_df=1,use_idf=True,ngram_range=(1,3))
tv_train_reviewsa=tv1.fit_transform(augmented_train_reviews)
tv_test_reviewsa=tv1.transform(augmented_test_reviews)


In [89]:
tv2=TfidfVectorizer(min_df=0.0,max_df=1,use_idf=True,ngram_range=(1,3))
tv_train_reviewsap=tv2.fit_transform(augmented_train_reviews_p)
tv_test_reviewsap=tv2.transform(augmented_test_reviews_p)


In [71]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [72]:
#Spliting the sentiment data
train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]
print(train_sentiments)
print(test_sentiments)

[[1]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [73]:
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bown=lr.fit(cv_train_reviewsn,train_sentiments)

print(lr_bown)
#Fitting the model for tfidf features
lr_tfidfn=lr.fit(tv_train_reviewsn,train_sentiments)

print(lr_tfidfn)


LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [74]:
lr1=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)

lr_bowa=lr1.fit(cv_train_reviewsa,train_sentiments)
print(lr_bowa)
lr_tfidfa=lr1.fit(tv_train_reviewsa,train_sentiments)
print(lr_tfidfa)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [90]:
lr2=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)

lr_bowap=lr2.fit(cv_train_reviewsap,train_sentiments)
print(lr_bowap)
lr_tfidfap=lr2.fit(tv_train_reviewsap,train_sentiments)
print(lr_tfidfap)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [91]:
#Predicting the model for bag of words
lr_bow_predictn=lr.predict(cv_test_reviewsn)
lr_bow_predicta=lr1.predict(cv_test_reviewsa)
lr_bow_predictap=lr2.predict(cv_test_reviewsap)

print(lr_bow_predictn)
print(lr_bow_predicta)

##Predicting the model for tfidf features
lr_tfidf_predictn=lr.predict(tv_test_reviewsn)
lr_tfidf_predicta=lr1.predict(tv_test_reviewsa)
lr_tfidf_predictap=lr2.predict(tv_test_reviewsap)

print(lr_tfidf_predictn)
print(lr_tfidf_predicta)

[0 0 0 ... 0 1 1]
[0 0 0 ... 0 1 1]
[0 0 0 ... 0 1 1]
[0 0 0 ... 0 1 0]


In [92]:
#Classification report for bag of words 
lr_bow_reportn=classification_report(test_sentiments,lr_bow_predictn,target_names=['Positive','Negative'])
lr_bow_reporta=classification_report(test_sentiments,lr_bow_predicta,target_names=['Positive','Negative'])
lr_bow_reportap=classification_report(test_sentiments,lr_bow_predictap,target_names=['Positive','Negative'])

print("Normal")
print(lr_bow_reportn)
print("Augemented")
print(lr_bow_reporta)
print("Augemented Polarity")
print(lr_bow_reportap)

#Classification report for tfidf features
lr_tfidf_reportn=classification_report(test_sentiments,lr_tfidf_predictn,target_names=['Positive','Negative'])
lr_tfidf_reporta=classification_report(test_sentiments,lr_tfidf_predicta,target_names=['Positive','Negative'])
lr_tfidf_reportap=classification_report(test_sentiments,lr_tfidf_predictap,target_names=['Positive','Negative'])

print("Normal")
print(lr_tfidf_reportn)
print("Augemented")
print(lr_tfidf_reporta)
print("Augemented Polarity")
print(lr_tfidf_reportap)

Normal
              precision    recall  f1-score   support

    Positive       0.75      0.75      0.75      4993
    Negative       0.75      0.75      0.75      5007

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000

Augemented
              precision    recall  f1-score   support

    Positive       0.74      0.75      0.75      4993
    Negative       0.75      0.74      0.75      5007

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000

Augemented Polarity
              precision    recall  f1-score   support

    Positive       0.75      0.75      0.75      4993
    Negative       0.75      0.75      0.75      5007

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75   

In [77]:
#training the linear svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)

#fitting the svm for bag of words
svm_bown=svm.fit(cv_train_reviewsn,train_sentiments)

print(svm_bown)

#fitting the svm for tfidf features
svm_tfidfn=svm.fit(tv_train_reviewsn,train_sentiments)

print(svm_tfidfn)


SGDClassifier(max_iter=500, random_state=42)
SGDClassifier(max_iter=500, random_state=42)


In [78]:
svm1=SGDClassifier(loss='hinge',max_iter=500,random_state=42)
svm_bowa=svm1.fit(cv_train_reviewsa,train_sentiments)
print(svm_bowa)
svm_tfidfa=svm1.fit(tv_train_reviewsa,train_sentiments)
print(svm_tfidfa)

SGDClassifier(max_iter=500, random_state=42)
SGDClassifier(max_iter=500, random_state=42)


In [79]:
#Predicting the model for bag of words
svm_bow_predictn=svm.predict(cv_test_reviewsn)
svm_bow_predicta=svm1.predict(cv_test_reviewsa)

print(svm_bow_predictn)
print(svm_bow_predicta)

#Predicting the model for tfidf features
svm_tfidf_predictn=svm.predict(tv_test_reviewsn)
svm_tfidf_predicta=svm1.predict(tv_test_reviewsa)

print(svm_tfidf_predictn)
print(svm_tfidf_predicta)

[1 1 0 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]


In [80]:
#Accuracy score for bag of words
svm_bow_scoren=accuracy_score(test_sentiments,svm_bow_predictn)
print("svm_bow_score :",svm_bow_scoren)
svm_bow_scorea=accuracy_score(test_sentiments,svm_bow_predicta)
print("svm_bow_score :",svm_bow_scorea)
#Accuracy score for tfidf features
svm_tfidf_scoren=accuracy_score(test_sentiments,svm_tfidf_predictn)
print("svm_tfidf_score :",svm_tfidf_scoren)
svm_tfidf_scorea=accuracy_score(test_sentiments,svm_tfidf_predicta)
print("svm_tfidf_score :",svm_tfidf_scorea)

svm_bow_score : 0.5829
svm_bow_score : 0.5863
svm_tfidf_score : 0.5112
svm_tfidf_score : 0.5112


In [81]:
#Classification report for bag of words 
svm_bow_reportn=classification_report(test_sentiments,svm_bow_predictn,target_names=['Positive','Negative'])
print(svm_bow_reportn)
svm_bow_reporta=classification_report(test_sentiments,svm_bow_predicta,target_names=['Positive','Negative'])
print(svm_bow_reporta)
#Classification report for tfidf features
svm_tfidf_reportn=classification_report(test_sentiments,svm_tfidf_predictn,target_names=['Positive','Negative'])
print(svm_tfidf_reportn)
svm_tfidf_reporta=classification_report(test_sentiments,svm_tfidf_predicta,target_names=['Positive','Negative'])
print(svm_tfidf_reporta)


              precision    recall  f1-score   support

    Positive       0.94      0.18      0.30      4993
    Negative       0.55      0.99      0.70      5007

    accuracy                           0.58     10000
   macro avg       0.74      0.58      0.50     10000
weighted avg       0.74      0.58      0.50     10000

              precision    recall  f1-score   support

    Positive       0.94      0.18      0.31      4993
    Negative       0.55      0.99      0.71      5007

    accuracy                           0.59     10000
   macro avg       0.74      0.59      0.51     10000
weighted avg       0.74      0.59      0.51     10000

              precision    recall  f1-score   support

    Positive       1.00      0.02      0.04      4993
    Negative       0.51      1.00      0.67      5007

    accuracy                           0.51     10000
   macro avg       0.75      0.51      0.36     10000
weighted avg       0.75      0.51      0.36     10000

              preci

0

In [93]:
os.system("echo -e '\a'")

0